In [1]:
pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import re
from collections import deque
import datetime
from wordcloud import WordCloud

## 1. 파일호출

In [3]:
# 파일 불러오기
with open('../../data/df_da.pkl', 'rb') as f:
    df_da = pickle.load(f)


In [4]:
# 파일 불러오기
with open('../../data/df_body.pkl', 'rb') as f:
    df_body = pickle.load(f)


In [5]:
df_da['date']   = df_da['q_creationdate'].dt.date
df_da['year']   = df_da['q_creationdate'].dt.year
df_da['month']  = df_da['q_creationdate'].dt.month
df_da['day']    = df_da['q_creationdate'].dt.day

In [6]:
df_body['date'] = df_body['creationdate'].dt.date
df_body['year'] = df_body['creationdate'].dt.year
df_body['month'] = df_body['creationdate'].dt.month
df_body['day'] = df_body['creationdate'].dt.day

In [7]:
df_body.head()

,creationdate,id,posttypeid,title,owneruserid,body,answered_yn,date,year,month,day
0,2023-10-26 15:34:53.830,77368423,2,None,508057.0,"<p>If all machines are on the same LAN, then y...",0,2023-10-26,2023,10,26
1,2023-11-01 08:52:42.577,77401389,1,Unable to do File Upload using sendkeys method...,22838353.0,<p>I am trying to automate a webpage using Sel...,0,2023-11-01,2023,11,1
2,2023-10-30 14:32:47.247,77389520,1,Run macro based on cell contents if cell changes,22828175.0,<p>I want to run a macro if a cell changes bas...,0,2023-10-30,2023,10,30
3,2023-10-30 14:36:20.843,77389551,2,None,12500315.0,<p>Whilst you <em>could</em> write your own wr...,0,2023-10-30,2023,10,30
4,2023-10-23 09:20:47.107,77344034,1,Asp.Net chunking cookie issue,3365923.0,<p>I am using ASP.NET Identity. The login app ...,0,2023-10-23,2023,10,23


## 2. Preprocessing 

In [8]:
# def cleanhtml(raw_html):
#   cleanr = re.compile('<(/)?([a-zA-Z0-9]*)(\\s[a-zA-Z0-9]*=[^>]*)?(\\s)*(/)?>')
#   cleantext = re.sub(cleanr, '', raw_html)
#   cleantext = re.sub(r"\n", "", cleantext)
#   cleantext = re.sub(r"\r", "", cleantext)
#   cleantext = re.sub(r"\t", "", cleantext)
#   return cleantext

In [9]:
def cleanhtml(raw_html):
  #1. replace newline to space
  # To analyze the text, it would be hard to understand all source code in python, so replace all the <code> tag first
  # src_re = re.compile('<code>(?=[^<]*\n)[\s\S]*?<\/code>')
  # src_re = re.compile('<code>[^<]*\\n[^<]*<\/code>')
  src_re = re.compile('<code>(.*?)<\/code>')
  cleantext_1 = re.sub(src_re, '', raw_html.replace('\n', '_**_'))
  cleantext_1 = cleantext_1.replace('_**_', '\n')

  tag_re = re.compile('<.*?>')
  cleantext_2 = re.sub(tag_re, '', cleantext_1)

  cleantext_2 = re.sub(r"\n", "", cleantext_2)
  cleantext_2 = re.sub(r"\r", "", cleantext_2)
  cleantext_2 = re.sub(r"\t", "", cleantext_2)

  return cleantext_2

In [10]:
df_body['body']

0           <p>If all machines are on the same LAN, then y...
1           <p>I am trying to automate a webpage using Sel...
2           <p>I want to run a macro if a cell changes bas...
3           <p>Whilst you <em>could</em> write your own wr...
4           <p>I am using ASP.NET Identity. The login app ...
                                  ...                        
43417231    <p>try this:</p>\n\n<p><div class="snippet" da...
43417232    <p>Hope this helps</p>\n\n<pre><code>num_list ...
43417233    <p>You can, for each 10 iterations, store the ...
43417234    <p>I am starting with a large dataframe. I wou...
43417235    <p>I have an overlay which appears on screen w...
Name: body, Length: 43417236, dtype: object

In [11]:
# apply the function, cleanhtml to the question and body text
df_body['body_prep'] = df_body['body'].apply(cleanhtml)

In [12]:
df_wl = df_body[['id', 'creationdate', 'body_prep']].copy()

In [13]:
def wordlen(text):
    return [x for x in text.split(' ') if len(x) > 6]

In [14]:
df_wl['over6'] = df_wl['body_prep'].apply(wordlen)

In [15]:
arr = []
df_wl['over6'].apply(lambda x : arr.extend(x))

0           None
1           None
2           None
3           None
4           None
            ... 
43417231    None
43417232    None
43417233    None
43417234    None
43417235    None
Name: over6, Length: 43417236, dtype: object

In [16]:
arr

['machines',
 'instructs',
 'connect',
 'address',
 'header,',
 'instead',
 'configured',
 'address',
 '(through',
 'Remember',
 'connection',
 'incoming',
 'debugging',
 'connections',
 'single-user',
 'connections.Requesting',
 'script/application',
 'through',
 'browser',
 'activated',
 '(either',
 'through',
 'browser',
 'extension)',
 'server,',
 'address',
 'machine',
 'initiated',
 'request.',
 'connect',
 'configured',
 'address',
 'machine',
 'running',
 'browser).If',
 'network,',
 'dbgpProxy',
 'PHP/Xdebug',
 'connect',
 'configure',
 'forward',
 'incoming',
 'request',
 'explaining',
 'greater',
 'detail:',
 'https://www.youtube.com/watch?v=_3RkGZK-UC8&amp;feature=youtu.beNeither',
 'dbgpProxy',
 'methods',
 'anything',
 'neither',
 'above-mentioned',
 'methods',
 'set-up,',
 'alternatively',
 'debugging',
 'complicated',
 'scenarios.',
 'automate',
 'webpage',
 'Selenium',
 'headless',
 'webpage',
 'buttons',
 '&quot;Choose',
 'File&quot;,',
 '&quot;Upload&quot;.',
 'type=

In [17]:
arr_str = ' '.join(arr)

In [ ]:
wordcloud = WordCloud().generate(arr_str)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
plt.imshow(wordcloud)   # wordcloud 객체를 넣으면 워드클라우드 형태의 그래프 생성
plt.axis('off')  #눈금 삭제 
plt.show()